In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/home/pt/test1/data/mnist", one_hot=True)

Extracting /home/pt/test1/data/mnist/train-images-idx3-ubyte.gz
Extracting /home/pt/test1/data/mnist/train-labels-idx1-ubyte.gz
Extracting /home/pt/test1/data/mnist/t10k-images-idx3-ubyte.gz
Extracting /home/pt/test1/data/mnist/t10k-labels-idx1-ubyte.gz


In [3]:

mb_size = 64
z_dim = 100
X_dim = mnist.train.images.shape[1]
y_dim = mnist.train.labels.shape[1]
h_dim = 128
c = 0
lr = 1e-3


def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig


def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)


# =============================== Q(z|X) ======================================

X = tf.placeholder(tf.float32, shape=[None, X_dim])
c = tf.placeholder(tf.float32, shape=[None, y_dim])
z = tf.placeholder(tf.float32, shape=[None, z_dim])

Q_W1 = tf.Variable(xavier_init([X_dim + y_dim, h_dim]))
Q_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

Q_W2_mu = tf.Variable(xavier_init([h_dim, z_dim]))
Q_b2_mu = tf.Variable(tf.zeros(shape=[z_dim]))

Q_W2_sigma = tf.Variable(xavier_init([h_dim, z_dim]))
Q_b2_sigma = tf.Variable(tf.zeros(shape=[z_dim]))


def Q(X, c):
    inputs = tf.concat(axis=1, values=[X, c])
    h = tf.nn.relu(tf.matmul(inputs, Q_W1) + Q_b1)
    z_mu = tf.matmul(h, Q_W2_mu) + Q_b2_mu
    z_logvar = tf.matmul(h, Q_W2_sigma) + Q_b2_sigma
    return z_mu, z_logvar


def sample_z(mu, log_var):
    eps = tf.random_normal(shape=tf.shape(mu))
    return mu + tf.exp(log_var / 2) * eps


# =============================== P(X|z) ======================================

P_W1 = tf.Variable(xavier_init([z_dim + y_dim, h_dim]))
P_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

P_W2 = tf.Variable(xavier_init([h_dim, X_dim]))
P_b2 = tf.Variable(tf.zeros(shape=[X_dim]))


def P(z, c):
    inputs = tf.concat(axis=1, values=[z, c])
    h = tf.nn.relu(tf.matmul(inputs, P_W1) + P_b1)
    logits = tf.matmul(h, P_W2) + P_b2
    prob = tf.nn.sigmoid(logits)
    return prob, logits


# =============================== TRAINING ====================================

z_mu, z_logvar = Q(X, c)
z_sample = sample_z(z_mu, z_logvar)
_, logits = P(z_sample, c)

# Sampling from random z
X_samples, _ = P(z, c)

# E[log P(X|z)]
recon_loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=X), 1)
# D_KL(Q(z|X) || P(z|X)); calculate in closed form as both dist. are Gaussian
kl_loss = 0.5 * tf.reduce_sum(tf.exp(z_logvar) + z_mu**2 - 1. - z_logvar, 1)
# VAE loss
vae_loss = tf.reduce_mean(recon_loss + kl_loss)

solver = tf.train.AdamOptimizer().minimize(vae_loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('out/'):
    os.makedirs('out/')

i = 0

for it in range(1000000):
    X_mb, y_mb = mnist.train.next_batch(mb_size)

    _, loss = sess.run([solver, vae_loss], feed_dict={X: X_mb, c: y_mb})

    if it % 1000 == 0:
        print('Iter: {}'.format(it))
        print('Loss: {:.4}'. format(loss))
        print()

        y = np.zeros(shape=[16, y_dim])
        for j in range(16):
            y[j, j%10] = 1
            
        samples = sess.run(X_samples,
                           feed_dict={z: np.random.randn(16, z_dim), c: y})

        fig = plot(samples)
        plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
plt.close(fig)

Iter: 0
Loss: 773.0

Iter: 1000
Loss: 144.0

Iter: 2000
Loss: 126.5

Iter: 3000
Loss: 109.5

Iter: 4000
Loss: 102.2

Iter: 5000
Loss: 115.0

Iter: 6000
Loss: 107.9

Iter: 7000
Loss: 108.4

Iter: 8000
Loss: 109.2

Iter: 9000
Loss: 109.3

Iter: 10000
Loss: 103.8

Iter: 11000
Loss: 104.3

Iter: 12000
Loss: 107.3

Iter: 13000
Loss: 103.6

Iter: 14000
Loss: 108.7

Iter: 15000
Loss: 109.2

Iter: 16000
Loss: 110.9

Iter: 17000
Loss: 101.5

Iter: 18000
Loss: 106.6

Iter: 19000
Loss: 107.9

Iter: 20000
Loss: 104.2



/root/miniconda3/envs/tensorflow/lib/python3.6/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Iter: 21000
Loss: 108.7

Iter: 22000
Loss: 103.8

Iter: 23000
Loss: 101.8

Iter: 24000
Loss: 101.8

Iter: 25000
Loss: 109.9

Iter: 26000
Loss: 101.2

Iter: 27000
Loss: 106.0

Iter: 28000
Loss: 104.5

Iter: 29000
Loss: 105.7

Iter: 30000
Loss: 104.8

Iter: 31000
Loss: 103.9

Iter: 32000
Loss: 103.7

Iter: 33000
Loss: 102.8

Iter: 34000
Loss: 105.0

Iter: 35000
Loss: 101.2

Iter: 36000
Loss: 102.4

Iter: 37000
Loss: 103.6

Iter: 38000
Loss: 103.3

Iter: 39000
Loss: 98.49

Iter: 40000
Loss: 107.0

Iter: 41000
Loss: 104.8



KeyboardInterrupt: 